# Example: Calculate Delta, Theta, Vega, Rho, and Gamma for Short and Long Options Contracts
[The Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) are quantitative measures that describe the sensitivity of an American option's price to various risk factors. Understanding and monitoring the Greeks is crucial for effective options trading and risk management. The key Greeks for American options include Delta, Gamma, Vega, Theta, and Rho, each quantifying how the option's value changes with movements in the underlying asset price, volatility, time decay, and interest rates.

> __Learning Objectives:__
>
> By the end of this example, you should be able to:
> * __Compute and interpret the five key Greeks for American options:__ Calculate Delta, Theta, Vega, Rho, and Gamma for both call and put options using the CRR binomial model, and understand what each Greek measures in terms of option price sensitivity.
> * __Compare Greek values between calls and puts:__ Recognize how the signs and magnitudes of Greeks differ between long call and long put positions, and understand the directional implications of each Greek for different contract types.
> * __Convert Greeks between long and short positions:__ Apply the principle that Greeks for long and short positions sum to zero to determine Greek values from the perspective of contract sellers versus buyers.

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> __Include:__ The [`include(...)` command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 

Let's set up our code environment:

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # include the Include.jl file

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

### Constants
Let's set a value for the step size `Δt,` i.e., `1 day` in units of years (assuming a `365-day` year), the risk-free rate at the time we gathered the data for the option contracts; we approximate this value using the [yield on the 10-year Treasury Note](https://ycharts.com/indicators/10_year_treasury_rate). We also set a value for the underlying share price `Sₒ` and the number of levels in the binomial tree model, `h`; we assume 6-levels per day.

In [2]:
Δt = (1/365); # options use 365 day year
risk_free_rate = 0.0431; # yield 10-year treasury note on 13-Jun-2024
Sₒ = 131.88; # after close on 14-Jun-2024 (close: 131.88)
h = 366; # 6-points per trading day

### Contract models
Next, we construct contract models for the options in this example. We begin by building an instance of the [MyAmericanCallContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanCallContractModel) using the [custom build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}). The [build(...) method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.build-Tuple{Type{MyAmericanCallContractModel},%20NamedTuple}) takes two arguments: the type to build, and a [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) containing the strike price `K`, the `premium` paid for the option, the days to expiration `DTE` (units: years), the `sense = 1` (long), the number of contracts in the `copy` field and the implied volatility in the `IV` field.

We save the call option contract model in the `call_option_model` variable:

In [3]:
call_option_model = build(MyAmericanCallContractModel, (
    K = 136.0, 
    premium = 9.23, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4848
));

The put contract, which is an instance of the [MyAmericanPutContractModel type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.MyAmericanPutContractModel), is constructed in a similar way to the call option. We save the put option contract model in the `put_option_model` variable:

In [4]:
put_option_model = build(MyAmericanPutContractModel, (
    K = 128.0, 
    premium = 7.78, # mark price
    DTE = (61.0)*Δt,
    sense = 1,
    copy = 1,
    IV = 0.4787
));

__Note__: In both cases, we construct long contract models, i.e., we look at option dynamics from the perspective of a contract buyer. To convert to a short perspective, i.e., from the perspective of the contract seller, we multiply the Greek values by `-1` (see Task 3).

___

## Task 1: Compute the Greeks for a Long Call Option
Let's begin our examination of the Greeks by calculating the Greek values for a long call option. From the buyer's viewpoint, a long call option is a substitute for owning shares. The buyer speculates that the underlying share price will go up during the contract and hopes to profit from the increase in the option contract's price. 

However, the option contract price is a function of many different market factors, not just the share price of the underlying asset. The Greeks provide a quantitative means of computing the contract price's sensitivity to changes in market conditions.

Let's start our exploration of the Greeks by computing `delta` for the long call.

### Delta Long Call
Delta measures the rate of change in the option's premium $\mathcal{P}_{\tau}$ with respect to a `+1 USD/share` change in the underlying asset's price.
* We estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta). The [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) takes several arguments: the `call_option_model` instance, and parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta) returns a value for Delta. We store the estimated value for Delta in the `delta_long_call::Float64` variable:

So what do we get?

In [5]:
delta_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the greek function
    greek_value = delta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.4920000000000009

__Summary__: For a long call, $\Delta_{c}\geq{0}$ implies that the option premium $\mathcal{P}_{c}$ increases with increasing share price. This underscores the use case of long call contracts as share replacements for underlying shares. 

> __Hedging Interpretation:__
>
> The $\Delta_{c}$ of a contract can be used to compute the equivalent number of shares that the option represents, denoted as $n_{c}$ shares per contract:
>$$
\begin{align*}
n_{c} &= 100\cdot\Delta_{c}
\end{align*}
$$
> where $\Delta_{c}$ denotes the $\Delta$ value for a long call contract. 

Thus, this long call contract is equivalent to:

In [6]:
println("Number of share equivalents: $(delta_long_call*100) shares per contract")

Number of share equivalents: 49.20000000000009 shares per contract


#### Check: How good is our delta estimate?
The delta value for the long call is $\Delta$ = `0.4979.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

So what do we find?

In [7]:
observed_value_delta_long_call = let
    observed_value_delta_long_call = 0.4979; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024 
    @assert isapprox(observed_value_delta_long_call, delta_long_call, atol=1e-2) # good to second decimal place, and sign is correct!
    observed_value_delta_long_call;
end;

### Theta Long Call
Theta measures the rate of change in the option's premium $\mathcal{P}_{\tau}$ for a `1-day` decrease in the time to maturity of the contract.
* We estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta). The [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta) takes several arguments: the `call_option_model` instance, and parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T.`
* The [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta) returns a value for Theta. We store the estimated value for Theta in the `theta_long_call::Float64` variable:

What is our time decay estimate?

In [8]:
theta_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the greek function
    greek_value = theta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.09199999999999875

__Summary__: For a long call, $\Theta_{c}<0$ implies that the option premium $\mathcal{P}_{c}$ decreases as we approach the expiration of the contract. Thus, all else being constant, a long call will lose value with time. This loss is referred to as the time decay of the long call contract. For the NVDA call contract, we have a time decay value of (units: USD per day per contract):

In [9]:
println("The long call changes in value: $(theta_long_call*100) USD per day per contract")

The long call changes in value: -9.199999999999875 USD per day per contract


#### Check: How good is our theta estimate?
This option's observed theta value is $\Theta$ = `-0.0937` (long call). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So what happens?

In [10]:
observed_value_theta_long_call = let
    observed_value_theta_long_call = -0.0937; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024
    @assert isapprox(observed_value_theta_long_call, theta_long_call, atol=1e-2) # good to second decimal place, and sign is correct!
    observed_value_theta_long_call;
end;

### Vega Long Call
Vega measures the rate of change in the option's premium $\mathcal{P}_{\tau}$ with respect to a `+1%` change in the underlying asset's implied volatility.
* We estimate $\nu$ using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega). The [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega) returns a value for Vega. We store the estimated value for vega in the `vega_long_call::Float64` variable:

So what is our volatility sensitivity estimate?

In [11]:
vega_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = vega(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.21600000000000108

__Summary__: For the long call contract, $\nu_{c}>0$. Thus, the option premium for this long call will increase with increasing implied volatility. In other words, as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive. This makes sense as the option seller demands additional compensation for the additional risk.

#### Check: How good is our vega estimate?
This vega value for the long call is $\nu$ = `0.2159.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So did we pass the test?

In [12]:
observed_value_vega_long_call = let
    observed_value_vega_long_call = 0.2159; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024
    @assert isapprox(observed_value_vega_long_call, vega_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_vega_long_call;
end;

### Rho Long Call
Rho measures the rate of change in the option's premium $\mathcal{P}_{\tau}$ for a `+1%` change in the risk-free rate $\bar{r}$. 
* We estimate $\rho$ using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho). The [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho) returns a value for $\rho$. We store the estimated value for $\rho$ in the `rho_long_call::Float64` variable:

So what is our interest rate sensitivity estimate?

In [13]:
rho_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = rho(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.0940000000000012

__Summary__: For a long call contract, $\rho_{c}>0$. Thus, the option premium increases with increasing risk-free rate. Under normal circumstances, the risk-free rate $\bar{r}$ doesn't significantly change. However, when interest rates increase during inflationary periods, a positive Rho shows that option premiums will increase, all else being equal.

In [14]:
println("Change in contract value by a 1% increase in risk-free rate: $(rho_long_call*100) USD per contract")

Change in contract value by a 1% increase in risk-free rate: 9.40000000000012 USD per contract


#### Check: How good is our rho estimate?
The $\rho$ value for the long call is $\rho$ = `0.0950.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values.

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So what happens?

In [15]:
observed_value_rho_long_call = let
    observed_value_rho_long_call = 0.0950; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024
    @assert isapprox(observed_value_rho_long_call, rho_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_rho_long_call;
end;

### Gamma Long Call
Gamma is the last common Greek we'll explore. The $\Gamma$ measures the rate of change in Delta for a `+1 USD/share` change in the underlying price.
* We estimate $\Gamma$ using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma). The [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) takes several arguments: the `call_option_model` instance, and several parameters used to compute the risk-neutral binomial share price tree, e.g., the number of tree levels `h,` the current share price  `Sₒ,` the number of days to expiration `T,` etc.
* The [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma) returns a value for $\Gamma$, which we store in the `gamma_long_call::Float64` variable:

What is our curvature estimate?

In [16]:
gamma_long_call = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = call_option_model; # alias the contract model

    # call the Greek function -
    greek_value = gamma(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.022999999999999687

__Summary__: The long call option has a $\Gamma_{c}>0$, which increases the premium's sensitivity to share price movements. The gamma value indicates how sensitive an option's delta is to changes in the underlying price, with higher gamma values indicating greater delta sensitivity.

#### Check: How good is our gamma estimate?
The gamma value for the long call is $\Gamma$ = `0.0152.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So do we pass the test?

In [17]:
observed_value_gamma_long_call = let
    observed_value_gamma_long_call = 0.0152; # NVDA 61-DTE K = 136 long call after close 14-Jun-2024
    @assert isapprox(observed_value_gamma_long_call, gamma_long_call, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_gamma_long_call;
end;

___

## Task 2: Compute the Greeks for a Long Put Option
Having computed all five Greeks for the long call contract, let's now examine how these values differ for a long put option. We'll compare the computed values with observed market data and contrast the results with those of the long call. Let's begin with Delta.

### Delta Long Put
Like the call option, we estimate $\Delta$ using the [delta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.delta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [18]:
delta_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = delta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.3879999999999999

__Summary__: The $\Delta_{p}<0$ for the long put (opposite sign of the call). Thus, as the underlying share price increases, the value of the put option decreases. This is consistent with the notion that long puts make a bearish directional assumption, i.e., a buyer will purchase a put option if they believe the underlying asset's share price will decrease. 

> __What about the number of share equivalents?__ 
> The number of share equivalents for a long put, denoted as $n_{p}$, will be negative. This means that we are short the shares but long the put. By being short the shares, we benefit from the decrease in the underlying price. For this long put, we have $n_{p}$ defined as:
> $$
n_{p} = 100\cdot\Delta_{p}
$$
> where $\Delta_{p}$ denotes the $\Delta$ value for a long put contract. 

Thus, this long put contract is equivalent to:

In [19]:
println("Number of share equivalents for our long put: $(delta_long_put*100) shares per contract")

Number of share equivalents for our long put: -38.79999999999999 shares per contract


#### Check: How good is our delta estimate?
The observed $\Delta$ value for the long put is $\Delta$ = `-0.3847` (long put). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values.

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated:

Is our estimate accurate?

In [20]:
observed_value_delta_long_put = let
    observed_value_delta_long_put = -0.3847; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
    @assert isapprox(observed_value_delta_long_put, delta_long_put, atol=1e-2) # good to second decimal place, and sign is correct!
    observed_value_delta_long_put;  
end;

### Theta
Like the call option, we estimate $\Theta$ using the [theta(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.theta), but in this case using the `put_option_model` instance, all other parameters held the same:

In [21]:
theta_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = theta(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

-0.07499999999999929

__Summary__: The $\Theta_{p}<0$ for the long put (same sign as the call option). Thus, the long call and the long put behave similarly in that the price of the option contract decreases as the contract approaches expiration, where all else is held constant.

In [22]:
println("The long put changes in value: $(theta_long_put*100) USD per day per contract")

The long put changes in value: -7.499999999999929 USD per day per contract


#### Check: How good is our theta estimate?
The observed theta value for the long put is $\Theta$ = `-0.0726` (long put). Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So are we good?

In [23]:
observed_value_theta_long_put = let
    observed_value_theta_long_put = -0.0726; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024
    @assert isapprox(observed_value_theta_long_put, theta_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_theta_long_put;
end;

### Vega
Like the call option, we estimate `Vega` using the [vega(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.vega), but in this case using the `put_option_model` instance, all other parameters held the same:

In [24]:
vega_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = vega(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.20700000000000074

__Summary__: For the long put contract, $\nu_{p}>0$. Thus, the option premium for the long put (like the long call) will increase with implied volatility; as the uncertainty in the future share price increases (measured by the implied volatility), an option contract becomes more expensive (both calls and puts). This makes sense as the option seller will demand additional compensation for the additional risk.

#### Check: How good is our vega estimate?
The observed $\nu$ value for the long put is $\nu$ = `0.2068.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values.

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

What do we find?

In [25]:
observed_value_vega_long_put = let
    observed_value_vega_long_put = 0.2068; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024
    @assert isapprox(observed_value_vega_long_put, vega_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_vega_long_put;
end;

### Rho
Like the call option, we estimate $\rho$ using the [rho(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.rho), but in this case using the `put_option_model` instance, all other parameters held the same:

In [26]:
rho_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = rho(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end;

__Summary__: For a long put contract, $\rho_{p}<0$. Thus, the option premium decreases with increasing risk-free rate. Under normal circumstances, the risk-free rate $\bar{r}$ doesn't significantly change. However, when interest rates increase during inflationary periods, a negative $\rho$ shows that put option premiums will decrease, all else being equal.

In [27]:
println("Change in put contract price by a 1% increase in risk-free rate: $(rho_long_put*100) USD per contract")

Change in put contract price by a 1% increase in risk-free rate: -8.29999999999993 USD per contract


#### Check: How good is our rho estimate?
The observed $\rho$ value for the long put is $\rho$ = `-0.0902.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values.

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

Are we close?

In [28]:
observed_value_rho_long_put = let
    observed_value_rho_long_put = -0.0902; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024 
    @assert isapprox(observed_value_rho_long_put, rho_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_rho_long_put;
end;

### Gamma
Finally, like the $\Gamma$ calculation for the long-call, we estimate $\Gamma$ using the [gamma(...) function](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/derivatives/#VLQuantitativeFinancePackage.gamma), but in this case using the `put_option_model` instance, all other parameters held the same:

In [29]:
gamma_long_put = let

    # initialize
    greek_value = 0.0; # initially set the greek value to 0
    contract_model = put_option_model; # alias the contract model

    # call the Greek function -
    greek_value = gamma(contract_model, h = h,  T = contract_model.DTE, 
        σ = contract_model.IV, Sₒ = Sₒ, μ = risk_free_rate);

    # return -
    greek_value;
end

0.020000000000000462

__Summary__: The long put option has a $\Gamma_{p}>0$, which increases the premium's sensitivity to share price movements, i.e., the premium becomes more responsive to changes in share price. The Gamma value indicates how sensitive an option's Delta is to changes in the underlying price, with higher Gamma values indicating greater Delta sensitivity.

#### Check: How good is our gamma estimate?
The observed gamma value for the long put is $\Gamma$ = `0.0150.` Let's use the [Julia @assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with the [Julia isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) to compare the computed and observed values. 

> __Test:__ The [isapprox(...) function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) compares two values and returns `true` if the two values are equal to some absolute or relative tolerance, in this case, specified by the absolute tolerance `atol` variable. If `false` (values are not equal), an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is generated.

So do we pass the test?

In [30]:
observed_value_gamma_long_put = let
    observed_value_gamma_long_put = 0.0150; # NVDA 61-DTE K = 128 long put after close 14-Jun-2024
    @assert isapprox(observed_value_gamma_long_put, gamma_long_put, atol=1e-2) # good to the second decimal place, and sign is correct!
    observed_value_gamma_long_put;
end;

___

## Task 3: Conversion of the Greeks to Short Contracts
We computed Delta, Theta, Vega, Rho, and Gamma values for long put and call contracts in the previous tasks. The question now is, what about short contracts? Computing the Greek values for short contracts is straightforward: the Greek values for long and short contracts must sum to zero.


> __Greeks for Short Contracts__
>
> Denote the short value of a Greek by an overbar, i.e., the $\Delta$ for a short call contract evaluated at market conditions would be given by $\bar{\Delta}_{c}$. Then, the following condition must hold (for all market states):
> $$
\begin{align*}
\bar{\Delta}_{c} + \Delta_{c} = 0
\end{align*}
$$

It's important to note that the interconversion between long and short Greek values is a universal concept applicable to all the Greeks we have explored. This understanding will serve as a solid foundation for your options trading knowledge.

Unhide the code block below to see how we constructed a table for the Greeks for the long call NVDA contract.

In [31]:
let
    
    sense = -1;
    greek_dataframe = DataFrame();

    # delta -
    delta_row = (
        greek = "delta", 
        call_long_calc = delta_long_call,
        call_short_calc = (sense)*delta_long_call,
        check = delta_long_call + (sense)*delta_long_call,
        call_long_obs = observed_value_delta_long_call,
        error_percentage = ((delta_long_call - observed_value_delta_long_call)/(observed_value_delta_long_call))*100
    )
    push!(greek_dataframe,delta_row);

    # theta
    theta_row = (
        greek = "theta", 
        call_long_calc = theta_long_call,
        call_short_calc = (sense)*theta_long_call,
        check = theta_long_call + (sense)*theta_long_call,
        call_long_obs = observed_value_theta_long_call,
        error_percentage = ((theta_long_call - observed_value_theta_long_call)/(observed_value_theta_long_call))*100,
    )
    push!(greek_dataframe,theta_row);

    # vega -
    vega_row = (
        greek = "vega", 
        call_long_calc = vega_long_call,
        call_short_calc = (sense)*vega_long_call,
        check = vega_long_call + (sense)*vega_long_call,
        call_long_obs = observed_value_vega_long_call,
        error_percentage = ((vega_long_call - observed_value_vega_long_call)/(observed_value_vega_long_call))*100
    )
    push!(greek_dataframe,vega_row);

    # rho -
    rho_row = (
        greek = "rho", 
        call_long_calc = rho_long_call,
        call_short_calc = (sense)*rho_long_call,
        check = rho_long_call + (sense)*rho_long_call,
        call_long_obs = observed_value_rho_long_call,
        error_percentage = ((rho_long_call - observed_value_rho_long_call)/(observed_value_rho_long_call))*100
    )
    push!(greek_dataframe,rho_row);

    # gamma -
    gamma_row = (
        greek = "gamma", 
        call_long_calc = gamma_long_call,
        call_short_calc = (sense)*gamma_long_call,
        check = gamma_long_call + (sense)*gamma_long_call,
        call_long_obs = observed_value_gamma_long_call,
        error_percentage = ((gamma_long_call - observed_value_gamma_long_call)/(observed_value_gamma_long_call))*100
    )
    push!(greek_dataframe,gamma_row);

    # display -
    pretty_table(
        greek_dataframe;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- ---------------- ----------------- --------- --------------- ------------------
   greek   call_long_calc   call_short_calc     check   call_long_obs   error_percentage 
  String          Float64           Float64   Float64         Float64            Float64 
 -------- ---------------- ----------------- --------- --------------- ------------------
   delta            0.492            -0.492       0.0          0.4979           -1.18498
   theta           -0.092             0.092       0.0         -0.0937            -1.8143
    vega            0.216            -0.216       0.0          0.2159          0.0463177
     rho            0.094            -0.094       0.0           0.095           -1.05263
   gamma            0.023            -0.023       0.0          0.0152            51.3158
 -------- ---------------- ----------------- --------- --------------- ------------------


and the NVDA put contract:

In [32]:
let
    
    sense = -1;
    greek_dataframe = DataFrame();

    # delta -
    delta_row = (
        greek = "delta", 
        put_long_calc = delta_long_put,
        put_short_calc = (sense)*delta_long_put,
        check = delta_long_put + (sense)*delta_long_put,
        put_long_obs = observed_value_delta_long_put,
        error_percentage = ((delta_long_put - observed_value_delta_long_put)/(observed_value_delta_long_put))*100
    )
    push!(greek_dataframe,delta_row);

    # theta
    theta_row = (
        greek = "theta", 
        put_long_calc = theta_long_put,
        put_short_calc = (sense)*theta_long_put,
        check = theta_long_put + (sense)*theta_long_put,
        put_long_obs = observed_value_theta_long_put,
        error_percentage = ((theta_long_put - observed_value_theta_long_put)/(observed_value_theta_long_put))*100,
    )
    push!(greek_dataframe,theta_row);

    # vega -
    vega_row = (
        greek = "vega", 
        put_long_calc = vega_long_put,
        put_short_calc = (sense)*vega_long_put,
        check = vega_long_put + (sense)*vega_long_put,
        put_long_obs = observed_value_vega_long_put,
        error_percentage = ((vega_long_put - observed_value_vega_long_put)/(observed_value_vega_long_put))*100
    )
    push!(greek_dataframe,vega_row);

    # rho -
    rho_row = (
        greek = "rho", 
        put_long_calc = rho_long_put,
        put_short_calc = (sense)*rho_long_put,
        check = rho_long_put + (sense)*rho_long_put,
        put_long_obs = observed_value_rho_long_put,
        error_percentage = ((rho_long_put - observed_value_rho_long_put)/(observed_value_rho_long_put))*100
    )
    push!(greek_dataframe,rho_row);

    # gamma -
    gamma_row = (
        greek = "gamma", 
        put_long_calc = gamma_long_put,
        put_short_calc = (sense)*gamma_long_put,
        check = gamma_long_put + (sense)*gamma_long_put,
        put_long_obs = observed_value_gamma_long_put,
        error_percentage = ((gamma_long_put - observed_value_gamma_long_put)/(observed_value_gamma_long_put))*100
    )
    push!(greek_dataframe,gamma_row);

    # display -
    pretty_table(
        greek_dataframe;
        backend = :text,
        fit_table_in_display_horizontally = false,
        table_format = TextTableFormat(borders = text_table_borders__compact)
    );
end

 -------- --------------- ---------------- --------- -------------- ------------------
   greek   put_long_calc   put_short_calc     check   put_long_obs   error_percentage 
  String         Float64          Float64   Float64        Float64            Float64 
 -------- --------------- ---------------- --------- -------------- ------------------
   delta          -0.388            0.388       0.0        -0.3847           0.857811
   theta          -0.075            0.075       0.0        -0.0726            3.30579
    vega           0.207           -0.207       0.0         0.2068          0.0967118
     rho          -0.083            0.083       0.0        -0.0902           -7.98226
   gamma            0.02            -0.02       0.0          0.015            33.3333
 -------- --------------- ---------------- --------- -------------- ------------------


__Summary__: In general, our estimates of [the Greeks](https://en.wikipedia.org/wiki/en:Greeks_(finance)?variant=zh-tw) have percentage error in the range of less than 10% for both the call and put contracts. 

> __Observation:__ The exception is $\Gamma$, with a percentage error above 30% in both cases (we overestimated the $\Gamma$ in each case). However, the $\Gamma$ value is small, and the sign was correctly captured, so while not ideal, the impact on the premium will be small.

That's curious about Gamma! Something to explore in future work.
___

## Summary
This example demonstrates how to compute and interpret the five key Greeks for American call and put options, and how to convert between long and short positions.

> __Key Takeaways:__
>
> * __Greeks quantify option price sensitivity:__ Delta measures sensitivity to underlying price changes, Theta captures time decay, Vega measures volatility sensitivity, Rho tracks interest rate sensitivity, and Gamma measures the rate of change in Delta. Each Greek provides specific information about how an option's value responds to market changes.
> * __Call and put Greeks have different signs:__ Long calls have positive Delta (benefit from price increases) while long puts have negative Delta (benefit from price decreases). Both long calls and long puts have negative Theta (lose value over time) and positive Vega (benefit from increased volatility). Rho is positive for calls and negative for puts.
> * __Long and short Greeks sum to zero:__ The Greek values for long and short positions in the same contract are equal in magnitude but opposite in sign, allowing straightforward conversion between buyer and seller perspectives by multiplying by negative one.

Understanding the Greeks enables better assessment of option positions and their exposure to different market risk factors.
___

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses should be used.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.